<br><br>
<p align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/d/df/Logo_UNIR.png" width="260"/>
</p>
<br><br><br>
<p align="center"><strong>Resolución de un problema mediante búsqueda heurística A</p>

<p align="center">Algoritmo A* para mover inventarios con un robot</strong></p>
<br><br><br>
<p align="center"><strong>Inteligencia Artificial<br></p>

<p align="center">Razonamiento y Planificación Automática <br></strong></p>
<br><br><br>
<p align="center"><strong>Equipo D-1033:</strong><br>   - Mayra Itzel Rodríguez <br>
              - César Alberto Tapia Mar <br>
              - Ramón Antonio García <br>
              - Victor M. Dominguez Garcia <br>
              <br><br><br>
Fecha: 28 de julio de 2025</p>
<br><br><br><br><br>

---



## 1. **Introducción**

El objetivo de esta actividad es implementar la estrategia de búsqueda heurística A* para resolver un problema real propuesto por la empresa Amazon, donde un robot debe mover tres inventarios a posiciones específicas dentro de un almacén representado como una matriz 4x4. El robot puede moverse horizontal y verticalmente, cargar y descargar inventarios, evitando obstáculos.

Utilizaremos la distancia de Manhattan como función heurística y un coste uniforme por cada acción. El resultado final será un plan de acciones que lleva al estado objetivo.



### Enunciado del problema (Parte 1)

"En esta actividad has de utilizar la estrategia de búsqueda heurística A* con el fin de generar un plan que permita al robot de Amazon mover el inventario de un estado inicial a un estado objetivo."


### Enunciado del problema (Parte 2)

El estado inicial del problema lo vamos a representar en una matriz de caracteres 4x4 de la siguiente manera

```
---------------------------------------------
|    M1    |     #    |          |    M3    |
---------------------------------------------
|          |     #    |          |          |
---------------------------------------------
|    M2    |          |     R    |          |
---------------------------------------------
|          |          |          |          |
---------------------------------------------
```

Donde:

* R: representa el robot. Inicialmente está ubicado en la posición [2,2]
* #: representa una pared.
* M1, M2 y M3: representan los tres inventarios que el robot debe mover. Y se encuentran ubicadas en las posiciones [0,0], [2,0] y [0,3] respectivamente.

El estado objetivo también será una matriz de 4x4 con las características definidas.



## 2. Desarrollo




La actividad la realizamos mediante iteraciones sucesivas que intentan mejorar la efectividad de la solución mediante entregables cada vez más complejos.

### Primer iteración

Siguiendo la parte 1 del enunciado del problema, iniciamos la ideación de la solución a través de la siguiente firma

```
a_estrella(inicio: Estado_Inicial) -> Estado_Objetivo
```

Esto es, buscamos una función que tome un estado inicial y como resultado nos entregue un estado objetivo. En este punto aún no sabemos los detalles de dichos estados, pero es suficiente para nuestra primer iteración.

## Segunda iteración

En esta segunda iteración buscaremos definir los parámetros de la clase "Estado". Tal como está descrito en la parte 2 del enunciado del problema, el estado deberá almacenar una matriz de 4x4, asi como una lista de posiciones de las paredes y una lista de posiciones de inventarios. Con esto, ahora nuestra segunda vuelta puede verse como

```
Posicion = TypeVar('Posicion)   #Posicion puede ser de cualquier tipo.

clase Estado:
    matriz: Lista[ Posicion ]
    paredes: Lista[ Posicion ]
    inventarios: Lista[ posicion]


a_estrella(inicio: Estado) -> fin: Estado
```

Por el momento lo único que sabemos de la clase Estado es que representará conjuntos de posiciones en la bodega, pero no estamos seguros de cúal será la representación computacional de estas posiciones.

## **Tercer iteración. Definicion e implementación de la clase Estado**

En esta iteración buscamos definir que significa Estado en el contexto de nuestro problema y los tipos de datos que usaremos para representarlo.


### Coordenadas

Dado que vamos a trabajar con mapas cuadrados de 4x4, creemos que tiene sentido definir cada nodo como una tupla. De esta manera, una posicion en el mapa se podrá representar por el par (x,y). Además, notamos que {'M1', 'M2', 'M3'} son ids para los inventarios.


In [ ]:
from typing import Tuple

Posicion = Tuple[int, int]
Id = str


Habiendo acordado una representación para las posiciones, ahora podemos definir las ubicaciones iniciales y finales de los inventarios como diccionarios que conectan los nombres con sus respectivas posiciones:
```
Inventarios_Inicio = {M1: (0,0), M2:(0,2), M3:(3,0)}
Inventarios_Objetivo = {M1:(3,3), M2:(3,2), M3:(3,1)}
```

In [ ]:
from typing import Dict

Inventario = Dict[Id, Posicion]

inventario_inicial: Inventario = { #Definición de posiciones de inicio para M1, M2 y M3
    'M1': (0, 0),
    'M2': (2, 0),
    'M3': (0, 3)
}

inventario_objetivo: Inventario = { #Definición de posiciones de objetivo para M1, M2 y M3
    'M1': (3, 3),
    'M2': (3, 2),
    'M3': (3, 1),
}

### **Igualdad entre estados**

Dado que un grafo solo depende de sus conjuntos A y B de nodos y vértices,también vale la pena que definamos el significado de la igualdad entre estados.

Diremos que dos estados E y F son iguales si sus entradas son iguales componente a componente:

```
E(e1,e2,...,ek) === F(f1,f2,...,fk)
si y solo si
e1 === f1 y
e2 === f2
...
ek === fk
```

De esta manera podremos construir la condicion de parada de nuestro algoritmo

```
Si
  Estado_Actual === Estado_Final
Entonces
  Fin
```


In [ ]:
def igualdad_estados(estado1, estado2):
    return (estado1.robot == estado2.robot and
            estado1.inventarios == estado2.inventarios)


class Estado_Test:
    def __init__(self, robot, inventarios):
        self.robot = robot
        self.inventarios = inventarios


estado_inicial = Estado_Test((2, 2), { 'M1': (0, 0), 'M2': (2, 0), 'M3': (0, 3)})
estado_objetivo = Estado_Test((2, 2),{'M1': (0, 0),'M2': (2, 0),'M3': (0, 3)})


assert igualdad_estados(estado_inicial, estado_objetivo) #Correcto

### **Implementación de la clase Estado**

En este punto ya estamos listos para implementar los métodos que acabamos de definir.

Se decidio que la clase debía contener la posicion del robot, la posicion de los paquetes a mover, y también un indicador de si el robot está cargando algun paquete en un momento dado.

La igualdad entre los estados se implementó de dos maneras: como el metodo es_objetivo() y como el método __eq__(). El primero es de uso particular y nos dirá si hemos finalizado mientras que el segundo es de uso general, necesario para definir el comportamiento del hash para la clase.  

In [ ]:

class Estado:
    def __init__(self, robot: Posicion, inventarios: Inventario, cargando=None):
        self.robot = robot  #(fila, columna)
        self.inventarios = inventarios.copy()  #{'M1': (fila, col)}
        self.cargando = cargando  #None o 'M1', 'M2', 'M3'

    def es_objetivo(self):
        for key in inventario_objetivo:
            if self.inventarios[key] != inventario_objetivo[key]:
                return False
        if self.cargando is not None: # Añadí esto para comprobar que el inventario está en el suelo
            return False

        return True

    def __eq__(self, other):
       return (self.robot == other.robot and
                self.inventarios == other.inventarios and
                self.cargando == other.cargando)

    def __hash__(self):
        return hash((self.robot, tuple(sorted(self.inventarios.items())), self.cargando))


## **Cuarta Iteración. Definición del mapa**

En pasos anteriores definimos lo que es una posición en el espacio de movimientos del robot y con ello definimos los conceptos de longitud, estado e igualdad de estados.

En esta iteración el objetivo ahora es imponer restricciones, tales como las dimensiones (en el entregable final debe ser de 4x4) y los obstaculos que el robot no puede atravesar (las paredes).

### **Paredes**

Ademas del estado, debemos definir el conjunto Pared constituido de las posiciones que el Robot no puede visitar.

Las paredes definidas por el problema son las siguientes:
```
---------------------------------------------
|          |     #    |          |          |
---------------------------------------------
|          |     #    |          |          |
---------------------------------------------
|          |          |          |          |
---------------------------------------------
|          |          |          |          |
---------------------------------------------
```
La construcción de nuestro algorítmo de busqueda parte de la idea de que el mapa inicial se puede representar a través de un grafo, de manera que sea posible construir la ruta del Robot como una selección de nodos y aristas.
```
0     0    0------0
|     |    |      |
|     |    |      |
0     0    0------0
|          |      |
|          |      |
0-----0----0------0
|     |    |      |
|     |    |      |
0-----0----0------0
```
En esta representación, las paredes son nodos que no están conectados al resto del grafo, y, por lo tanto, no se pueden estudiar como caminos. En este sentido, las paredes siguen siendo listas de posiciones en el mapa.

In [ ]:
from typing import List

Pared: List[Posicion] = [
    (0, 1), (1, 1)
 ] #Nodos no unidos al resto del grafo


### **Heurística y movimientos**

Lo que diferencía al algorítmo A* de otras formas de busqueda como Dijkstra, es la incorporación de una heurística que nos permitirá elegir los caminos mas prometedores.



### Distancia Manhattan

La distancia al objetivo genera una buena heurística siempre y cuando sea admisible, esto es, que la distancia calculada no sea mayor al número de pasos para llegar al objetivo. Para el caso de nuestro mapa formado por posiciones discretas y con un robot que solo se puede mover en dos dimensiones, la distancia euclidianda puede resultar incongruente.

Definimos la distancia Manhattan segun:

```
  d(a,b) = d( (x1,y1), (x2,y2) ) = |x2 - x1| + |y2 - y1|
```
Esta definicion de distancia es util en mapas cuadrados porque es computacionalmente eficiente y además porque forma una heurística admisible.

In [ ]:
def manhattan(p1: Posicion, p2: Posicion) -> int:
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])


assert manhattan((0, 0), (1, 1)) == 2 #correcto
assert manhattan((0, 0), (0, 0)) == 0 #correcto
assert manhattan((0, 0), (2, 3)) == 5 #correcto

Usando esta definición podemos calcular la distancia entre el estado inicial y el estado objetivo.

```
Si M_k es la posición inicial del inventario k y si N_k es la posición final del inventario k
d(M1, N1) = d((0,0),(3,3)) = 6
d(M2, N2) = d((2,0),(3,2)) = 3
d(M3, N3) = d((0,3),(3,1)) = 5
distancia total = 6 + 3 + 5 = 14
```

In [ ]:
def heuristica(e: Estado) -> int:
  '''
  Toma el estado actual y devuelve la suma de las distancias manhattan de cada inventario a su posicion objetivo
  '''
  inventario: Inventario = e.inventarios
  h = 0
  for id, pos in inventario.items():
    if pos != inventario_objetivo[id]:
      h += manhattan(pos, inventario_objetivo[id])
  return h

heuristica_test = Estado((2, 2), { 'M1': (0, 0), 'M2': (2, 0), 'M3': (0, 3)})
assert heuristica(heuristica_test) == 14 #correcto

### **Movimientos**

Las funciones límite y pared nos ayudarán a definir si nuestro robot aún se encuentra dentro del mapa o si no está intentando moverse a través de una pared.

In [ ]:
def limites(pos: Posicion) -> bool:
    return 0 <= pos[0] < 4 and 0 <= pos[1] < 4

assert limites((0, 0)) #correcto
assert limites((4, 0)) == False #correcto
assert limites((0, 4)) == False #correcto

In [ ]:
def pared(pos: Posicion) -> bool:
    return pos in Pared

assert pared((0, 0)) == False #correcto
assert pared((0, 1)) #correcto

La función posición describe las direcciones en las que puede moverse el robot.

In [ ]:
def movimientos(pos: Posicion) -> Posicion:
      movimientos = [(-1, 0), (1, 0), (0, -1), (0, 1)] #arriba, abajo, izq, derecha
      nueva_pos = []
      for x, y in movimientos:
        nx, ny = pos[0] + x, pos[1] + y
        if limites((nx, ny)) and not pared((nx, ny)):
          nueva_pos.append((nx, ny))
      return nueva_pos


assert movimientos((0, 0)) == [(1,0)] #correcto
assert movimientos((3,3))  == [(2,3), (3,2)] #correcto


In [ ]:
def cargar_inventario(e: Estado):
    cargando = e.cargando
    if cargando is None:
        for k, v in e.inventarios.items():
            if v == e.robot:
                return Estado(e.robot, e.inventarios, k)
    return e # Return the original state if robot is already carrying something.


estado_test = Estado((0, 0), { 'M1': (0, 0), 'M2': (2, 0), 'M3': (0, 3)})
result = cargar_inventario(estado_test)
assert result.robot == (0, 0)
assert result.inventarios == { 'M1': (0, 0), 'M2': (2, 0), 'M3': (0, 3)}
assert result.cargando == 'M1'
assert result.es_objetivo() == False

In [ ]:
def sucesores(e: Estado):
    acciones = []
    robot: Posicion = e.robot
    inventario_actual: Inventario = e.inventarios
    cargando_actual = e.cargando

    # Movimiento del Robot (con o sin inventario)
    for movimiento_destino in movimientos(robot):
        costo_accion = 1

        # Caso 1: Robot no está cargando nada (se mueve solo)
        if cargando_actual is None:
            # El robot puede moverse a cualquier celda válida, incluso si hay un inventario
            # en esa celda, porque podría querer cargarlo.
            nuevo_estado = Estado(movimiento_destino, inventario_actual, None)
            acciones.append((f"Mover R a {movimiento_destino[0], movimiento_destino[1]}", nuevo_estado, costo_accion))

        # Caso 2: Robot está cargando un inventario (se mueve con él)
        else:
            inventario_cargado_nombre = cargando_actual

            # VERIFICACIÓN DE COLISIÓN AL MOVER INVENTARIO CARGADO
            # La nueva posición (movimiento_destino) no debe estar ocupada por OTRO inventario.
            colision_con_otro_inv = False
            for otro_id, otra_pos in inventario_actual.items():
                if otro_id != inventario_cargado_nombre and otra_pos == movimiento_destino:
                    colision_con_otro_inv = True
                    break # Colisión detectada, este movimiento no es posible

            if not colision_con_otro_inv: # Solo se procede si no hay colisión
                nuevos_inventarios_mov = inventario_actual.copy()
                nuevos_inventarios_mov[inventario_cargado_nombre] = movimiento_destino # El inventario se mueve con el robot
                nuevo_estado_con_inv = Estado(movimiento_destino, nuevos_inventarios_mov, inventario_cargado_nombre)
                acciones.append((f"Mover {inventario_cargado_nombre} a {movimiento_destino[0], movimiento_destino[1]}", nuevo_estado_con_inv, costo_accion))

    # Cargar Inventario
    if cargando_actual is None: # Solo puede cargar si no está cargando ya
        estado_despues_cargar = cargar_inventario(e)
        if estado_despues_cargar is not e: # Si realmente se cargó algo
            acciones.append((f"Cargar R: {estado_despues_cargar.cargando}", estado_despues_cargar, 0))

    # Descargar Inventario
    else: # Si el robot está cargando algo
        inventario_a_soltar_nombre = cargando_actual
        posicion_donde_dejar = robot # Se deja en la posición actual del robot

        # VERIFICACIÓN DE COLISIÓN AL DESCARGAR INVENTARIO
        # No se puede dejar un inventario en una celda ya ocupada por OTRO inventario.
        colision_al_dejar = False
        for otro_id, otra_pos in inventario_actual.items():
            if otro_id != inventario_a_soltar_nombre and otra_pos == posicion_donde_dejar:
                colision_al_dejar = True
                break # Colisión detectada, no se puede dejar aquí

        if not colision_al_dejar: # Solo se procede si no hay colisión
            nuevos_inventarios_descarga = inventario_actual.copy()
            nuevos_inventarios_descarga[inventario_a_soltar_nombre] = posicion_donde_dejar # El inventario se coloca en la posición del robot
            nuevo_estado_descargado = Estado(posicion_donde_dejar, nuevos_inventarios_descarga, None)
            acciones.append((f"Descargar R: {inventario_a_soltar_nombre}", nuevo_estado_descargado, 0))

    return acciones



## 4. Implementación del algoritmo A*


A*  es un algoritmo eficiente para encontrar el camino más corto dentro de un grafo, utilizando la exploración de nodos mediante una función f(n). Esta función combina el costo de llegar moverse hacia un nodo con una estimación del costo faltante para llegar al objetivo. Gracias a esto dar prioridad a la exploración de caminos prometedores. La fórmula general es:

```
f(n)=g(n)+h(n)
```
donde
- g(n): Representa el costo acumulado desde el estado inicial hasta el nodo actual n. Según la sección de tareas pedidas, cada acción del robot tiene un costo de 1 unidad. Por lo tanto, g(n) representa cuantos pasos o acciones hay que ejecutar para mover al robot del estado n hasta el estado n el estado inicial.

- h(n) Es una estimación del costo de mover al robot desde el nodo actual n hasta el nodo objetivo. Para implementar esta función se utilizó la Distancia Manhattan. Para un estado dado, h(n) se calcula como la suma de las distancias Manhattan de cada inventario (M1, M2, M3) desde su posición actual en el estado n hasta su posición objetivo final. Esta heurística es admisible porque nunca sobreestima el costo real para alcanzar el objetivo.

Una vez calculada una prioridad f para una elemento de la frontera, utilizamos una cola de prioridad, que en python devuelve el elemento más pequeño de un conjunto. Es así como el robot va siguiendo el camino que genera el estimado mas pequeño.


In [ ]:
import heapq

from itertools import count
contador = count()

def a_estrella(inicial: Estado):
    limite = []
    heapq.heappush(limite, (heuristica(inicial), next(contador), 0, inicial, []))
    visitados = {estado_inicial:0}

    while limite:
        _, _, costo_acumulado, estado, plan = heapq.heappop(limite)

        if estado.es_objetivo():
            return plan

        for accion, sucesor, costo_accion in sucesores(estado):
            if sucesor not in visitados or costo_acumulado + costo_accion < visitados[sucesor]:
                visitados[sucesor] = costo_acumulado + costo_accion
                costo_acumulado += costo_accion
                costo_accion = costo_acumulado + heuristica(sucesor)
                heapq.heappush(limite, (costo_accion, next(contador), costo_acumulado, sucesor, plan + [accion]))

    return None



## 5. Ejecución y resultado


Presentamos una serie de pruebas que muestran las acciones del robot, empezando por el caso trivial e incrementando su complejidad hasta probar el objetivo de la tarea.

In [ ]:
def print_plan(plan):
    pasos_movimiento = []
    if plan is None:
        print("No se encontró un plan.")
        return

    print("El plan es:\n")
    for i, paso in enumerate(plan):
        print(f"{i+1}. {paso}")
        if "Mover R a" in paso or "Mover M" in paso:
            pasos_movimiento.append(paso)

    print(f"\nEl total de pasos de movimiento es: {len(pasos_movimiento)}")
    print(f"El total de acciones es: {len(plan)}")


## Test 1

El estado inicial es identico al estado final. La distancia recorrida deberia ser cero, en tanto que los inventarios ya estan en su lugar.

```
---------------------------------------------
|          |          |          |          |
---------------------------------------------
|          |     #    |          |          |
---------------------------------------------
|          |     #    |     R    |          |
---------------------------------------------
|          |     M3   |     M2   |     M1   |
---------------------------------------------

Distancia al objetivo = 0
```

In [ ]:
posicion_robot = (2, 2)
inventarios = {
    'M1': (3, 3),
    'M2': (3, 2),
    'M3': (3, 1)
}
estado_inicial = Estado(
    posicion_robot,
    inventarios,
    cargando=None
)
plan = a_estrella(estado_inicial)
print_plan(plan)

El plan es:


El total de pasos de movimiento es: 0
El total de acciones es: 0


## Test 2

Hay un único inventario fuera de lugar. La distancia al objetivo debería ser de dos pasos, mientras que el número de acciones debería ser 6.

```
---------------------------------------------
|          |          |          |          |
---------------------------------------------
|          |     #    |          |          |
---------------------------------------------
|          |     #    |     R    |    M1    |
---------------------------------------------
|          |     M3   |     M2   |          |
---------------------------------------------

Distancia al objetivo = 2
```

In [ ]:
posicion_robot = (2, 2)
inventarios = {
    'M1': (2, 3),
    'M2': (3, 2),
    'M3': (3, 1)
}

estado_inicial = Estado(
    posicion_robot,
    inventarios,
    cargando=None
)

plan= a_estrella(estado_inicial)
print_plan(plan)

El plan es:

1. Mover R a (2, 3)
2. Cargar R: M1
3. Mover M1 a (3, 3)
4. Descargar R: M1

El total de pasos de movimiento es: 2
El total de acciones es: 4


## Test 3

Nuevamente hay un inventario fuera de lugar pero esta vez hay varios caminos posibles.
```
---------------------------------------------
|          |          |          |          |
---------------------------------------------
|          |     #    |          |     R    |
---------------------------------------------
|          |     #    |          |          |
---------------------------------------------
|    M3    |          |     M2   |    M1    |
---------------------------------------------

Distancia correcta al objetivo = 6
```

In [ ]:
posicion_robot = (1, 3)
inventarios = {
    'M1': (3, 3),
    'M2': (3, 2),
    'M3': (3, 0)
}

estado_inicial = Estado(
    posicion_robot,
    inventarios,
    cargando=None
)

plan = a_estrella(estado_inicial)

print_plan(plan)

El plan es:

1. Mover R a (2, 3)
2. Mover R a (3, 3)
3. Mover R a (3, 2)
4. Mover R a (3, 1)
5. Mover R a (3, 0)
6. Cargar R: M3
7. Mover M3 a (3, 1)
8. Descargar R: M3

El total de pasos de movimiento es: 6
El total de acciones es: 8


## Test 4. Estado Objetivo

In [ ]:
posicion_robot = (2, 2)
inventarios = {
    'M1': (0, 0),
    'M2': (2, 0),
    'M3': (0, 3)
}

estado_inicial = Estado(
    posicion_robot,
    inventarios,
    cargando=None
)

plan = a_estrella(estado_inicial)
print_plan(plan)

El plan es:

1. Mover R a (2, 1)
2. Mover R a (2, 0)
3. Cargar R: M2
4. Mover M2 a (3, 0)
5. Mover M2 a (3, 1)
6. Mover M2 a (3, 2)
7. Descargar R: M2
8. Mover R a (2, 2)
9. Mover R a (2, 1)
10. Mover R a (2, 0)
11. Mover R a (1, 0)
12. Mover R a (0, 0)
13. Cargar R: M1
14. Mover M1 a (1, 0)
15. Mover M1 a (2, 0)
16. Mover M1 a (2, 1)
17. Mover M1 a (2, 2)
18. Mover M1 a (2, 3)
19. Mover M1 a (3, 3)
20. Descargar R: M1
21. Mover R a (2, 3)
22. Mover R a (1, 3)
23. Mover R a (0, 3)
24. Cargar R: M3
25. Mover M3 a (1, 3)
26. Mover M3 a (2, 3)
27. Mover M3 a (2, 2)
28. Mover M3 a (2, 1)
29. Mover M3 a (3, 1)
30. Descargar R: M3

El total de pasos de movimiento es: 24
El total de acciones es: 30


## 6. Conclusiones y dificultades encontradas

La implementación del algoritmo de búsqueda heurística A* permitió resolver de manera eficiente el problema de planificación de movimientos para un robot en un entorno con restricciones, simulando un caso práctico como el de un almacén automatizado de Amazon. A través de la representación del espacio de estados en una matriz 4x4, se pudo modelar tanto la movilidad del robot como las acciones de carga y descarga de inventario, considerando también los obstáculos fijos.

Algunas de las dificultades que encontramos fueron el representar los estados con precisión para detectar repetidos, especialmente por la estructura mutable de los diccionarios, ajustar la heurística para garantizar eficiencia y completitud sin sobreestimar.
Otra dificultad inicial residió en la necesidad de generalizar y comparctar la función *sucesores*. La clave para resolver esto fue la reorganización de la lógica para manejar de mejor manera los mivimientos del robot (solo o con inventario) y las acciones del mismo (cargar o dejar), lo que resultó en un código más legible y eficiente.
Otro reto importante fue la gestión de las colisiones de inventarios. Asegurar que el robot no intentara mover o dejar el inventario en una celda donde ya existiera uno. La inclusión de verificaciones de colisión garantizó que solo se generaran estados válidos.
Una dificultad sutil, pero determinante para la selección de la mejor ruta fue la interpretación de los costos de las acciones. Inicialmente, el algoritmo asumía un costo uniforme de 1 por cada acción. lo que nos llevó a una solución que, aunque eran óptimos bajo esa métrica, no reflejaban la estrategia más eficiente e intuitiva. Esto nos llevó a introducir costos variables: Asignando un costo de 1 a los movimientos y un costo de 0 a las acciones de cargar y dejar.
Finalmente, la definición del estado objetivo también presentó un matiz importante para obtener un resultado más completo y satisfactorio. En un inicio, el algoritmo solo verificaba las posiciones finales de los inventarios, lo que permitía que el robot *terminara* la busqueda aun cargando un inventario en la posición final. Aunque esto no era incorrecto, no se sentía como si realmente se hubiera terminado el ejercicio. La adición de la condición *self.cargado is None* en *es_objetivo()*  nos permitió darle un cierre convincente al ejercicio.



## 7. Referencias

[1] Aplicación de heurísticas en caminos óptimos en gridworlds
Felner, A., Stern, R., Netanyahu, N. S., Kraus, S., & Holte, R. C. (2011).
Additive Pattern Database Heuristics.<br>
[2] Journal of Artificial Intelligence Research, 22, 279–318.
Analiza múltiples heurísticas, incluyendo Manhattan, y su efecto en el rendimiento de A* en entornos de tipo cuadrícula.<br>
[3] Libro sobre algoritmos de planificación
LaValle, S. M. (2006).<br>
Planning Algorithms. Cambridge University Press.
Disponible online: http://planning.cs.uiuc.edu/
[4] Python Software Foundation. (2024). Python documentation. https://docs.python.org/3/<br>
[5] Russell, S., & Norvig, P. (2010). *Artificial Intelligence: A Modern Approach* (3rd ed.). Prentice Hall.<br>
[6] Universidad Internacional de La Rioja (UNIR). (2024). *Algoritmos de búsqueda informada*. Aula virtual.


